In [0]:
%run /eReport/PkgGeneral/PrcGeneral

In [0]:
import pandas as pd
from datetime import datetime
from datetime import timedelta

class PRC_RCD_SOBREENDEUDAMIENTO():
  
  def __init__(self, id_solicitud):
      self.id_solicitud = id_solicitud
      
      #Obteniendo parametros
      parametros = PrcGeneral(id_solicitud = self.id_solicitud).obtenerParametros()
      self.empresa = parametros['empresa']  
      self.fecha_proceso = parametros['fecha_proceso']  
      self.fecha_proceso_str = self.fecha_proceso.strftime('%Y-%m-%d') 
      
      #Fecha de actualizacion
      fecha_actualizacion = datetime.now()
      self.fecha_actualizacion_str = fecha_actualizacion.strftime("%Y-%m-%d %H:%M:%S")
      
      #Conexion
      connection = PrcGeneral().databaseConnect()
      self.connection = connection
      
      cursor = self.connection.cursor()
      self.cursor = cursor
      
  def ProcesaCondicionSE(self):
    #Reproceso
    limpiaCondicionSE = """UPDATE CL
    SET CL.CONDICION_SOBRE_ENDEUDAMIENTO = '0'
    FROM CLIENTE CL
    WHERE  CL.FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
    AND CL.CONDICION_SOBRE_ENDEUDAMIENTO = '1'
    """
    self.cursor.execute(limpiaCondicionSE)
    self.connection.commit()
    
    #Proceso OBS: Validar si el cod_deudor es el codigo deudor sbs o el codigo deudor interno de la entidad
    actualizaCondicionSE = """UPDATE CL
    SET CL.CONDICION_SOBRE_ENDEUDAMIENTO = D.VALOR
    FROM BF_SOBREENDEUDAMIENTO D
    JOIN CLIENTE_DEUDOR CD
    ON CD.CODIGO_DEUDOR_INTERNO = D.COD_DEUDOR 
    JOIN CLIENTE CL
    ON CL.CLIENTE_DEUDOR = CD.ID
    AND CL.FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
    """
    
    self.cursor.execute(actualizaCondicionSE)
    self.connection.commit()
    
#PRC_RCD_SOBREENDEUDAMIENTO = PRC_RCD_SOBREENDEUDAMIENTO(1)
#PRC_RCD_SOBREENDEUDAMIENTO = PRC_RCD_SOBREENDEUDAMIENTO.ProcesaCondicionSE()